In [4]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import datetime as dt
import mplfinance as mpf
import yfinance as yf

In [10]:
#pip install yfinance

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\py\venv\scripts\python.exe -m pip install --upgrade pip' command.


# **Function to save stock data to CSV**


In [7]:
def save_to_csv_from_yahoo(ticker, sdate, edate):
  #start = dt.datetime(sdate)
  #end = dt.datetime(edate)

  #tickerData = yf.Ticker(ticker)
  
  # get historical market data
  #df = tickerData.history(period="max")
  yfdf = yf.download(ticker, start=sdate, end=edate)
  #yfdf = yf.download('gbtc', start="2021-01-01", end="2021-01-01")
  #print(yfdf)
  mydf = pd.DataFrame(yfdf)
  

  #yfdf.actions.to_csv('D:\\py\\pycode\\banas-tut01\\' + ticker + '.csv')
  #csvresult = mydf.to_csv('D:\\py\\pycode\\banas-tut01\\' + ticker + '.csv')
  #csvResult = df.to_csv("d:\\gbtc.csv")
  mydf.to_csv('d:/' + ticker + '.csv')
  #csvresult = mydf.to_csv('gbtc.csv')
  
  return mydf

In [ ]:
 #save_to_csv_from_yahoo("gbtc", "2021-01-01", "2021-11-03")

# **Get data from CSV**

In [12]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv('d:/' + ticker + '.csv')
    except FileNotFoundError:
        print("File not found")
    else:
        return df


# **Add daily return to DF**

In [15]:
# simple daily return: (Ending Prid - Beginning Price)/Beginning Price or (EP/BP)-1

def add_daily_return_to_df(df,ticker):
    df['Daily_Return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    df.to_csv('d:/' + ticker + '.csv')
    return df


# **Test Functions**

In [16]:
GBTC = get_df_from_csv("GBTC")

add_daily_return_to_df(GBTC,"GBTC")

GBTC

,Date,Open,High,Low,Close,Adj Close,Volume,Daily_Return
0,2021-01-04,34.090000,36.599998,32.900002,35.080002,35.080002,22817800,NaN
1,2021-01-05,34.490002,39.540001,34.490002,39.410000,39.410000,19229000,0.123432
2,2021-01-06,41.419998,44.000000,39.750000,43.930000,43.930000,40502900,0.114692
3,2021-01-07,47.700001,48.410000,40.810001,44.970001,44.970001,32619300,0.023674
4,2021-01-08,47.849998,48.650002,44.360001,44.419998,44.419998,23125700,-0.012230
...,...,...,...,...,...,...,...,...
206,2021-10-27,46.669998,47.720001,46.169998,46.700001,46.700001,8355200,-0.055230
207,2021-10-28,48.700001,49.000000,46.320000,48.685001,48.685001,5163600,0.042505
208,2021-10-29,48.560001,50.000000,48.230000,49.689999,49.689999,4692100,0.020643
209,2021-11-01,49.279999,49.540001,48.590000,49.240002,49.240002,4111100,-0.009056
